In [47]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# 1) Load PyBaMM parameters and grab OCP functions
param = pybamm.ParameterValues("Chen2020")          # "Chen2020" or "Marquis2019", "Mohtat2020"
ocv_anode   = param["Negative electrode OCP [V]"]  # Graphite
ocv_cathode = param["Positive electrode OCP [V]"]  # NCM811



# 3) Interactive update function
soc_an_fun = lambda soc: soc  # Anode SOC depletion function
soc_ca_fun = lambda soc: 1 - soc  # Cathode SOC depletion function
def update(soc, ocv_anode, ocv_cathode,soc_an_fun, soc_ca_fun,np_ratio=1,lamne=0,lampe=0):
    # 2) Build a fine SOC grid and compute OCV and differential capacity dQ/dV
    soc_grid = np.linspace(0, 1, 500)

    # Anode (graphite)
    V_an = ocv_anode(soc_grid)
    dVdQ_an = np.gradient(V_an, soc_grid)
    dQdV_an = 1.0 / dVdQ_an

    # Cathode (NCM811)
    V_ca = ocv_cathode(soc_grid)
    dVdQ_ca = np.gradient(V_ca, soc_grid)
    dQdV_ca = 1.0 / dVdQ_ca 

    # Get the half cell socs
    soc_an = soc_an_fun(soc)  # anode depletes
    soc_ca = soc_ca_fun(soc)   # cathode depletes

    # Ensure SOC is within bounds
    # Anode
    soc_an = np.clip(soc_grid, 0.0, soc)
    V_an_cur = ocv_anode(soc_an)
    dQdV_an_cur = np.interp(soc_an, soc_grid, dQdV_an)* (1 - lamne)* np_ratio
    #Cathode
    soc_ca = np.clip(soc_grid, 0.0, 1-soc)
    V_ca_cur = ocv_cathode(soc_ca)
    dQdV_ca_cur = np.interp(soc_ca, soc_grid, dQdV_ca)* (1 - lampe)
    # Interpolate current differential capacity and voltage

    # Plot
    #fig, ax = plt.subplots(figsize=(6, 5))
    fig, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot differential capacity curves    
    battery_color={"gray": "#333333",
                       "bright_gray": "#E7E6E6",
                        "gray_blue": "#44546A",
                        "navy_blue": "#003B73",
                        "light_blue":"#5DADE2",
                        "orange": "#F39C12",
                        "green": "#70AD47"}
    ax.plot(dQdV_an* (1 - lamne)*np_ratio, -V_an, label="Graphite tank", color=battery_color["gray"])
    ax.plot(dQdV_ca* (1 - lampe), -V_ca, label="NCM tank", color=battery_color["light_blue"])
    ax.plot(-dQdV_an* (1 - lamne)*np_ratio, -V_an, color=battery_color["gray"])
    ax.plot(-dQdV_ca* (1 - lampe), -V_ca, color=battery_color["light_blue"])
    # Plot BoL curves
    ax.plot(dQdV_an, -V_an*np_ratio, color=battery_color["gray"],linestyle='--')
    ax.plot(dQdV_ca, -V_ca, color=battery_color["light_blue"],linestyle='--')
    ax.plot(-dQdV_an, -V_an*np_ratio, color=battery_color["gray"],linestyle='--')
    ax.plot(-dQdV_ca, -V_ca, color=battery_color["light_blue"],linestyle='--')

    # Fill according to SOC
    ax.fill_betweenx(
        -V_an_cur, 
        dQdV_an_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )
    ax.fill_betweenx(
        -V_an_cur, 
        -dQdV_an_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )
    ax.fill_betweenx(
        -V_ca_cur, 
        -dQdV_ca_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )
    ax.fill_betweenx(
        -V_ca_cur, 
        dQdV_ca_cur,
        0,
        color=battery_color["navy_blue"], alpha=0.8,
    )

    # Add voltage difference arrow between anode and cathode at x=0
    ax.annotate(
        '',
        xy=(0, -V_ca_cur.min()), 
        xytext=(0, -V_an_cur .min()),
        arrowprops=dict(arrowstyle='<->', color='orange', linewidth=2)
    )
    mid_y = 0.5*( -V_ca_cur.min() -V_an_cur.min() )
    delta_V = V_ca_cur.min() - V_an_cur.min()


    ax.text(0.1, mid_y, f'ΔV = {delta_V:.3f} V', color='orange', va='center')
    # Labels and legend
    ax.set_xlabel("dQ/dV [Ah/V]")
    ax.set_ylabel("Voltage [V]")
    ax.legend(loc="right")
    ax.grid(True)
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 0)
    yticks = np.arange(-5, 1, 1)   # e.g. [-5, -4, -3, -2, -1, 0]
    ax.set_yticks(yticks)

    # 2) Label them with their absolute values
    ax.set_yticklabels([abs(int(t)) for t in yticks])

    # 3) Move legend to the left inside the axes
    ax.legend(loc="center left")
    #ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{abs(x):.2f}"))


    # 4) Plot the OCV curves
    soc_an_all = soc_an_fun(soc_grid)
    soc_ca_all = soc_ca_fun(soc_grid)
    V_an_all = ocv_anode(soc_an_all)
    V_ca_all = ocv_cathode(soc_ca_all)
    V_cell_all = V_ca_all - V_an_all
    ax2.plot(soc_grid, V_cell_all, color=battery_color["navy_blue"], label="Cell OCV")
    ax2.plot(soc, delta_V, color=battery_color["navy_blue"], marker='o', markersize=5, label="Current SOC")
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 5)
    ax2.grid(True)
    ax2.set_xlabel("State of Charge (SOC)")

    plt.show()
    plt.tight_layout()


# 4) Create slider and display widget
soc_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01, description="SOC"
)
update_reduce = lambda soc: update(soc, ocv_anode=ocv_anode,ocv_cathode=ocv_cathode,soc_an_fun=soc_an_fun, soc_ca_fun=soc_ca_fun,lamne=0, lampe=0)
widgets.interact(update_reduce, soc=soc_slider)


interactive(children=(FloatSlider(value=0.5, description='SOC', max=1.0, step=0.01), Output()), _dom_classes=(…

<function __main__.<lambda>(soc)>

In [22]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets


# ===== 1. Constants & Parameters =====
BATTERY_COLORS = {
    "gray": "#333333",
    "bright_gray": "#E7E6E6",
    "gray_blue": "#44546A",
    "navy_blue": "#003B73",
    "light_blue": "#5DADE2",
    "orange": "#F39C12",
    "green": "#70AD47"
}

param = pybamm.ParameterValues("Chen2020")
ocv_anode = param["Negative electrode OCP [V]"]
ocv_cathode = param["Positive electrode OCP [V]"]

soc_an_fun = lambda soc: soc
soc_ca_fun = lambda soc: 1 - soc


# ===== 2. OCV and dQ/dV Computation =====
def compute_ocv_and_dqdv(ocv_func, soc_grid):
    voltages = ocv_func(soc_grid)
    dVdQ = np.gradient(voltages, soc_grid)
    dQdV = 1.0 / dVdQ
    return voltages, dQdV


# ===== 3. Plotting Helpers =====
def plot_dqdv_tanks(ax, soc, ocv_anode, ocv_cathode, dQdV_an, dQdV_ca,
                    V_an, V_ca, np_ratio, lamne, lampe):
    # Get current SOC slices
    soc_grid = np.linspace(0, 1, 500)
    soc_an = np.clip(soc_grid, 0.0, soc)
    soc_ca = np.clip(soc_grid, 0.0, 1 - soc)

    V_an_cur = ocv_anode(soc_an)
    dQdV_an_cur = np.interp(soc_an, soc_grid, dQdV_an) * (1 - lamne) * np_ratio

    V_ca_cur = ocv_cathode(soc_ca)
    dQdV_ca_cur = np.interp(soc_ca, soc_grid, dQdV_ca) * (1 - lampe)

    # Plot symmetric dQ/dV tanks (BoL and aged)
    for sign in [1, -1]:
        ax.plot(sign * dQdV_an * (1 - lamne) * np_ratio, -V_an,
                color=BATTERY_COLORS["gray"], label="Graphite tank" if sign == 1 else None)
        ax.plot(sign * dQdV_ca * (1 - lampe), -V_ca,
                color=BATTERY_COLORS["light_blue"], label="NCM tank" if sign == 1 else None)
        ax.plot(sign * dQdV_an, -V_an * np_ratio,
                color=BATTERY_COLORS["gray"], linestyle='--')
        ax.plot(sign * dQdV_ca, -V_ca,
                color=BATTERY_COLORS["light_blue"], linestyle='--')

    # Fill current SOC region
    for dQdV, V in [(dQdV_an_cur, V_an_cur), (dQdV_ca_cur, V_ca_cur)]:
        for sign in [1, -1]:
            ax.fill_betweenx(-V, sign * dQdV, 0,
                             color=BATTERY_COLORS["navy_blue"], alpha=0.8)

    # Voltage delta arrow
    ax.annotate('', xy=(0, -V_ca_cur.min()), xytext=(0, -V_an_cur.min()),
                arrowprops=dict(arrowstyle='<->', color='orange', linewidth=2))
    delta_V = V_ca_cur.min() - V_an_cur.min()
    mid_y = -0.5 * (V_ca_cur.min() + V_an_cur.min())
    ax.text(0.1, mid_y, f'ΔV = {delta_V:.3f} V',
            color=BATTERY_COLORS["orange"], va='center')

    # Axes formatting
    ax.set_xlabel("dQ/dV [Ah/V]")
    ax.set_ylabel("Voltage [V]")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 0)
    yticks = np.arange(-5, 1, 1)
    ax.set_yticks(yticks)
    ax.set_yticklabels([abs(int(t)) for t in yticks])
    ax.legend(loc="center left")
    ax.grid(True)


def plot_ocv_curve(ax2, soc_grid, soc, ocv_anode, ocv_cathode, soc_an_fun, soc_ca_fun):
    soc_an_all = soc_an_fun(soc_grid)
    soc_ca_all = soc_ca_fun(soc_grid)
    V_cell = ocv_cathode(soc_ca_all) - ocv_anode(soc_an_all)

    delta_V = ocv_cathode(soc_ca_fun(soc)) - ocv_anode(soc_an_fun(soc))

    ax2.plot(soc_grid, V_cell, color=BATTERY_COLORS["navy_blue"], label="Cell OCV")
    ax2.plot(soc, delta_V, color=BATTERY_COLORS["navy_blue"], marker='o', markersize=5, label="Current SOC")
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 5)
    ax2.set_xlabel("State of Charge (SOC)")
    ax2.grid(True)
    ax2.legend()


# ===== 4. Main Update Function =====
def update_plot(soc, ocv_anode, ocv_cathode, soc_an_fun, soc_ca_fun, np_ratio=1, lamne=0, lampe=0):
    soc_grid = np.linspace(0, 1, 500)
    V_an, dQdV_an = compute_ocv_and_dqdv(ocv_anode, soc_grid)
    V_ca, dQdV_ca = compute_ocv_and_dqdv(ocv_cathode, soc_grid)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    plot_dqdv_tanks(ax1, soc, ocv_anode, ocv_cathode, dQdV_an, dQdV_ca,
                    V_an, V_ca, np_ratio, lamne, lampe)
    plot_ocv_curve(ax2, soc_grid, soc, ocv_anode, ocv_cathode, soc_an_fun, soc_ca_fun)

    plt.tight_layout()
    plt.show()


# ===== 5. Interactive Widget Setup =====
def launch_widget():
    soc_slider = widgets.FloatSlider(
        value=0.5, min=0.0, max=1.0, step=0.01, description="SOC"
    )
    update_func = lambda soc: update_plot(
        soc, ocv_anode, ocv_cathode, soc_an_fun, soc_ca_fun, lamne=0.5, lampe=0
    )
    return widgets.interact(update_func, soc=soc_slider)


# ===== 6. Run the Widget =====
launch_widget()


interactive(children=(FloatSlider(value=0.5, description='SOC', max=1.0, step=0.01), Output()), _dom_classes=(…

<function __main__.launch_widget.<locals>.<lambda>(soc)>

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pybamm


class BatteryTankPlotter:
    def __init__(
        self,
        ocv_anode,
        ocv_cathode,
        soc_an_fun=lambda soc: soc,
        soc_ca_fun=lambda soc: 1 - soc,
        np_ratio=1.0,
        v_min=None,
        v_max=None,
        lamne=0.0,
        lampe=0.0,
        resolution=500,
    ):
        self.ocv_anode = ocv_anode
        self.ocv_cathode = ocv_cathode
        self.soc_an_fun = soc_an_fun
        self.soc_ca_fun = soc_ca_fun
        self.np_ratio = np_ratio
        self.v_min = v_min
        self.v_max = v_max
        self.lamne = lamne
        self.lampe = lampe
        self.resolution = resolution

        self.colors = {
            "gray": "#333333",
            "bright_gray": "#E7E6E6",
            "gray_blue": "#44546A",
            "navy_blue": "#003B73",
            "light_blue": "#5DADE2",
            "orange": "#F39C12",
            "green": "#70AD47",
        }

    def _compute_ocv_and_dqdv(self, ocv_func):
        soc_grid = np.linspace(0, 1, self.resolution)
        voltage = ocv_func(soc_grid)
        dVdQ = np.gradient(voltage, soc_grid)
        dQdV = 1.0 / dVdQ
        return soc_grid, voltage, dQdV

    def _plot_voltage_limit_lines(self, ax, soc_grid):
        lines = []

        if self.v_min is not None:
            # anode at SOC=1 (fully lithiated), cathode at SOC=1 (fully lithiated)
            V_an1 = self.ocv_anode(self.soc_an_fun(1))
            V_ca0 = self.ocv_cathode(self.soc_ca_fun(1))
            lines.append(("an1", -V_an1))
            lines.append(("cath0", -V_ca0))

        if self.v_max is not None:
            # anode at SOC=0 (delithiated), cathode at SOC=0 (delithiated)
            V_an0 = self.ocv_anode(self.soc_an_fun(0))
            V_ca1 = self.ocv_cathode(self.soc_ca_fun(0))
            lines.append(("an0", -V_an0))
            lines.append(("cath1", -V_ca1))

        for label, y in lines:
            ax.axhline(y=y, color=self.colors["green"], linestyle=":", linewidth=1)
            ax.text(
                x=4.6, y=y, s=label, va="center", ha="right", fontsize=9, color=self.colors["green"]
            )

    def _plot_dqdv_tanks(self, ax, soc, soc_grid, V_an, dQdV_an, V_ca, dQdV_ca):
        soc_an = np.clip(soc_grid, 0.0, soc)
        soc_ca = np.clip(soc_grid, 0.0, 1 - soc)

        V_an_cur = self.ocv_anode(soc_an)
        V_ca_cur = self.ocv_cathode(soc_ca)

        dQdV_an_cur = np.interp(soc_an, soc_grid, dQdV_an) * (1 - self.lamne) * self.np_ratio
        dQdV_ca_cur = np.interp(soc_ca, soc_grid, dQdV_ca) * (1 - self.lampe)

        # Plot tanks
        for sign in [1, -1]:
            ax.plot(sign * dQdV_an * (1 - self.lamne) * self.np_ratio, -V_an,
                    color=self.colors["gray"], label="Graphite tank" if sign == 1 else None)
            ax.plot(sign * dQdV_ca * (1 - self.lampe), -V_ca,
                    color=self.colors["light_blue"], label="NCM tank" if sign == 1 else None)

            ax.plot(sign * dQdV_an* self.np_ratio, -V_an ,
                    color=self.colors["gray"], linestyle='--')
            ax.plot(sign * dQdV_ca, -V_ca,
                    color=self.colors["light_blue"], linestyle='--')

        # Fill tanks
        for dQdV, V in [(dQdV_an_cur, V_an_cur), (dQdV_ca_cur, V_ca_cur)]:
            for sign in [1, -1]:
                ax.fill_betweenx(-V, sign * dQdV, 0,
                                 color=self.colors["navy_blue"], alpha=0.8)

        # Voltage difference arrow
        delta_V = V_ca_cur.min() - V_an_cur.min()
        mid_y = -0.5 * (V_ca_cur.min() + V_an_cur.min())
        ax.annotate('', xy=(0, -V_ca_cur.min()), xytext=(0, -V_an_cur.min()),
                    arrowprops=dict(arrowstyle='<->', color=self.colors["orange"], linewidth=2))
        ax.text(0.1, mid_y, f'ΔV = {delta_V:.3f} V',
                color=self.colors["orange"], va='center')

        # Add voltage limit lines if set
        self._plot_voltage_limit_lines(ax, soc_grid)

        # Axes formatting
        ax.set_xlabel("dQ/dV [Ah/V]")
        ax.set_ylabel("Voltage [V]")
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 0)
        yticks = np.arange(-5, 1, 1)
        ax.set_yticks(yticks)
        ax.set_yticklabels([abs(int(t)) for t in yticks])
        ax.legend(loc="center left")
        ax.grid(True)

    def _plot_ocv_curve(self, ax, soc, soc_grid):
        soc_an = self.soc_an_fun(soc_grid)
        soc_ca = self.soc_ca_fun(soc_grid)

        V_an = self.ocv_anode(soc_an)
        V_ca = self.ocv_cathode(soc_ca)
        V_cell = V_ca - V_an

        delta_V = self.ocv_cathode(self.soc_ca_fun(soc)) - self.ocv_anode(self.soc_an_fun(soc))

        if self.v_min is not None and self.v_max is not None and self.v_max > self.v_min:
            # Interpolate SoC values corresponding to Vmin and Vmax
            soc_vs_voltage = lambda V_target: np.interp(V_target, V_cell, soc_grid)

            soc_vmin = soc_vs_voltage(self.v_min)
            soc_vmax = soc_vs_voltage(self.v_max)

            if soc_vmax == soc_vmin:  # Avoid divide by zero
                soc_norm = soc_grid * 0
                soc_point = 0
            else:
                soc_norm = (soc_grid - soc_vmin) / (soc_vmax - soc_vmin)
                soc_point = (soc - soc_vmin) / (soc_vmax - soc_vmin)

            ax.set_xlim(0, 1)
            ax.set_xlabel("Normalized SoC (0 at Vmin, 1 at Vmax)")
        else:
            soc_norm = soc_grid
            soc_point = soc
            ax.set_xlim(0, 1)
            ax.set_xlabel("State of Charge (SOC)")

        ax.plot(soc_norm, V_cell, color=self.colors["navy_blue"], label="Cell OCV")
        ax.plot(soc_point, delta_V, color=self.colors["navy_blue"], marker='o', markersize=5, label="Current SOC")
        ax.set_ylim(0, 5)
        ax.set_ylabel("Voltage [V]")
        ax.grid(True)
        ax.legend()


    def plot(self, soc=0.5):
        soc_grid, V_an, dQdV_an = self._compute_ocv_and_dqdv(self.ocv_anode)
        _, V_ca, dQdV_ca = self._compute_ocv_and_dqdv(self.ocv_cathode)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        self._plot_dqdv_tanks(ax1, soc, soc_grid, V_an, dQdV_an, V_ca, dQdV_ca)
        self._plot_ocv_curve(ax2, soc, soc_grid)

        plt.tight_layout()
        plt.show()


In [53]:
param = pybamm.ParameterValues("Chen2020")
ocv_anode = param["Negative electrode OCP [V]"]
ocv_cathode = param["Positive electrode OCP [V]"]

plotter = BatteryTankPlotter(
    ocv_anode,
    ocv_cathode,
    np_ratio=1.2,
    v_min=2.8,
    v_max=4.2,
    lamne=0,
    lampe=0,
)


def launch_widget():
    soc_slider = widgets.FloatSlider(
        value=0.5, min=0.0, max=1.0, step=0.01, description="SOC"
    )
    update_func = lambda soc: plotter.plot(soc)
    return widgets.interact(update_func, soc=soc_slider)



# ===== 6. Run the Widget =====
launch_widget()



interactive(children=(FloatSlider(value=0.5, description='SOC', max=1.0, step=0.01), Output()), _dom_classes=(…

<function __main__.launch_widget.<locals>.<lambda>(soc)>

In [46]:
ocv_anode(0.5)

0.1330855128593352